This notebook contains code that adds a "Near Burn Ratio" band to a geotiff. This could potentially be useful for detecting certain forest distirbances, particularly slash and burn agriculture.

In [1]:
import earthpy.spatial as es
import rasterio as rio
import glob

In [2]:
# Test glob function on folder containing geotiffs

glob.glob('D:/canopy_data/s2cloudless_new_model/null/*.tif')[2]

'D:/canopy_data/s2cloudless_new_model/null\\100_1000_1000.tif'

In [17]:
# Try a single test geotiff first

base_url = 'D:/canopy_data/s2cloudless_new_model/'

test_tif = glob.glob(base_url + 'null/*.tif')[2]

# open with rasterio
with rio.open(test_tif) as src:
    chip = src.read()
    meta = src.meta
    print(chip.shape)
    
nbr_band = es.normalized_diff(chip[8], chip[10]) # use normalized differential to create NBR band

meta.update(count = chip.shape[0] + 1) # add 1 to the number of bands in metadata

(12, 131, 131)


In [18]:
meta

{'driver': 'GTiff',
 'dtype': 'float64',
 'nodata': None,
 'width': 131,
 'height': 131,
 'count': 13,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(8.983152841195215e-05, 0.0, 19.14848859629172,
        0.0, -8.983152841195215e-05, -3.097660594229346)}

In [20]:
with rio.open(test_tif) as src:
    print(src.descriptions)

('B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12')


In [15]:
with rio.open('new_test.tif', 'w', **meta) as dst:
    with rio.open(test_tif) as src:
        for i in range(chip.shape[0]):
            dst.write_band(i+1, src.read(i+1)) # write all the old bands first
    dst.write_band(chip.shape[0]+1, nbr_band) # then write the NBR band

In [16]:
import os

In [25]:
def make_nbr_band(src_url, dst_url):
    """
    Inserts an NBR band into a geotiff
    src_url: The location of your geotiff
    dst_url: Where you want to save your new geotiff (includes filename)
    """
    with rio.open(src_url) as src:
        tif = src.read()
        meta = src.meta
        descriptions = src.descriptions
        
        nbr_band = es.normalized_diff(chip[8], chip[10]) # make NBR band
        meta.update(count=chip.shape[0]+1) # add one band to metadata
        
        with rio.open(dst_url, 'w', **meta) as dst:
            for i in range(chip.shape[0]):
                dst.write_band(i+1, src.read(i+1)) # write all the old bands
            dst.write_band(chip.shape[0]+1, nbr_band) # write the NBR band
            new_descriptions = descriptions + ('NBR',) # update descriptions
            dst.descriptions = new_descriptions

In [26]:
make_nbr_band(test_tif, 'new_test.tif') # test function

In [27]:
test_tif.split('\\')[-1]

'100_1000_1000.tif'

In [28]:
# Use function on all geotiffs in all relevant folders

folders = ['SAB', 'ISL', 'industrial_agriculture', 'null']

for folder in folders:
    geotiffs = glob.glob(base_url + folder + '/*.tif') # gets list of geotiffs in the folder
    for tif in geotiffs:
        filename = tif.split('\\')[-1] # pull out filename
        dst_folder = base_url + folder + '_nbr/' # update folder name for new geotiffs
        if not os.path.exists(dst_folder):
            os.mkdir(dst_folder)
        dst_url = dst_folder + filename
        make_nbr_band(tif, dst_url) # run function